# Load The Test Data

In [ ]:
import zipfile
import os
import shutil

dataset_zip = 'dataset/public_testing_dataset_1.zip'
dataset_dir = 'dataset/public_testing_dataset_1'

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

In [ ]:
!pwd

# Preprocess The Image to match the model input

The label_img folder contains images `.png`, and all the images does not contains the ground truth, so we need to add the fake paired images, and concatenate to the original images to create the paired images.

In [ ]:
import os

# source path
dataset_label_img_dir = 'dataset/public_testing_dataset_1/label_img'
# destination path
dataset_test_dir = 'dataset/public_testing_dataset_1/test'

# create destination directory if not exist
if not os.path.exists(dataset_test_dir):
	os.makedirs(dataset_test_dir)

We also have to convert the images from:
- black to white
- white to black

In [ ]:
# process each image in the source directory
# and copy to destination directory
import cv2
import numpy as np

for filename in os.listdir(dataset_label_img_dir):

	# read the image
	img = cv2.imread(os.path.join(dataset_label_img_dir, filename))

	# invert the color
	img = cv2.bitwise_not(img)

	# concatenate the image with itself horizontally
	img_double = np.concatenate((img, img), axis=1)

	# write the new image to the destination directory
	cv2.imwrite(os.path.join(dataset_test_dir, filename), img_double)

print('Done!')

# Copy the test folder to the model test input folder

In [ ]:
import os
import shutil

source_dir = './dataset/public_testing_dataset_1/test'
target_dir = './AI cup demo code/pytorch-CycleGAN-and-pix2pix/datasets/competition_test/test'

# If the target directory exists, remove it
if os.path.exists(target_dir):
    shutil.rmtree(target_dir)

# Copy the source directory to the target directory
shutil.copytree(source_dir, target_dir)

# Test The Model

In [ ]:
# change directory to the pix2pix code
import os
os.chdir('AI cup demo code/pytorch-CycleGAN-and-pix2pix')

In [ ]:
! python test.py --dataroot ./datasets/competition_test --name ROAD_pix2pix --model pix2pix --direction AtoB --num_test 1000

In [ ]:
print(f"Size: {len(os.listdir('./results/ROAD_pix2pix/test_latest/images/'))}")

# Transform the results into AI CUP format

The results are stored in `./ROAD_pix2pix/test_latest/images/`.

And there are 2 types of results: `real_A` and `fake_B`.

The `real_A` is the original image, and the `fake_B` is the transformed image.

Store the `{Prefix}_fake_B.png` as `{Prefix}.jpg` to `./ROAD_pix2pix/test_latest/submission/`.

In [ ]:
# store the fake images to the `./results/ROAD_pix2pix/test_latest/submission/` folder
import os
import cv2
import shutil

source_folder = './results/ROAD_pix2pix/test_latest/images'
target_folder = './results/ROAD_pix2pix/test_latest/submission'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for image_name in os.listdir(source_folder):
    if 'fake' in image_name:
        new_name = image_name.replace('_fake_B.png', '.jpg')
        shutil.copy(os.path.join(source_folder, image_name), os.path.join(target_folder, new_name))

In [ ]:
# resize the image as 420x240
import os
import cv2

for image_name in os.listdir(target_folder):
    img = cv2.imread(os.path.join(target_folder, image_name))
    img = cv2.resize(img, (420, 240))
    cv2.imwrite(os.path.join(target_folder, image_name), img)

In [ ]:
print(f"Size: {len(os.listdir(target_folder))}")
# zip the fake images
shutil.make_archive(target_folder, 'zip', target_folder)